### Load The Data

In [1]:
import pandas as pd
import numpy as np

In [2]:
df=pd.read_table('SMSSpamCollection',header=None,encoding='utf-8')

In [3]:
df.head()

0                                                  1
0   ham  Go until jurong point, crazy.. Available only ...
1   ham                      Ok lar... Joking wif u oni...
2  spam  Free entry in 2 a wkly comp to win FA Cup fina...
3   ham  U dun say so early hor... U c already then say...
4   ham  Nah I don't think he goes to usf, he lives aro...

In [4]:
df.shape

(5572, 2)

In [5]:
for i in range(0,5572):
    if df.iloc[i][0]=='ham':
        df.iloc[i][0]=0 
    else:
        df.iloc[i][0]=1 


In [6]:
df.head()

0                                                  1
0  0  Go until jurong point, crazy.. Available only ...
1  0                      Ok lar... Joking wif u oni...
2  1  Free entry in 2 a wkly comp to win FA Cup fina...
3  0  U dun say so early hor... U c already then say...
4  0  Nah I don't think he goes to usf, he lives aro...

In [7]:
df.shape

(5572, 2)

In [8]:
processed=df[1]
label=df[0]

### NLP Processing

In [9]:
df[1]

0       Go until jurong point, crazy.. Available only ...
1                           Ok lar... Joking wif u oni...
2       Free entry in 2 a wkly comp to win FA Cup fina...
3       U dun say so early hor... U c already then say...
4       Nah I don't think he goes to usf, he lives aro...
                              ...                        
5567    This is the 2nd time we have tried 2 contact u...
5568                 Will ü b going to esplanade fr home?
5569    Pity, * was in mood for that. So...any other s...
5570    The guy did some bitching but I acted like i'd...
5571                           Rofl. Its true to its name
Name: 1, Length: 5572, dtype: object

In [10]:
  processed=df[1]

In [11]:
processed=processed.str.replace(r'^\w+@[a-zA-Z_]+?\.[a-zA-Z]{2,3}$','emailaddr')###remove email
processed=processed.str.replace(r'^(http(s?)\:\/\/)*[0-9a-zA-Z]([-.\w]*[0-9a-zA-Z])*(:(0-9)*)*(\/?)([a-zA-Z0-9\-\.\?\,\'\/\\\+&amp;%\$#_]*)?$','url')###remove url
processed=processed.str.replace(r'^\s*(?:\+?(\d{1,3}))?[-. (]*(\d{3})[-. )]*(\d{3})[-. ]*(\d{4})(?: *x(\d+))?\s*$','mobilenum')###remove mobile number
processed=processed.str.replace(r'\d+','')
processed=processed.str.replace(r'[^\w\d\s]',' ')###remove space in between words
processed=processed.str.replace(r'\s+',' ')###remove multiple white spaces by a single white space
processed=processed.str.replace(r'^\s+|\s+?$','')

In [12]:
processed=processed.str.lower()

In [13]:
###remove accented words
import unicodedata
def remove_accented_chars(text):
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return text

In [14]:
s=remove_accented_chars(processed[0])
processed[0]=s

In [15]:
for i in range(0,len(processed)):
    s=remove_accented_chars(str(processed[i]))
    processed[i]=s

In [16]:
###stopwords
from nltk.corpus import stopwords

stop_words=set(stopwords.words('english'))

In [17]:
processed=processed.apply(lambda x: ' '.join(word for word in x.split() if word not in stop_words))

In [18]:
###Stemming
import nltk
ps=nltk.PorterStemmer()

processed=processed.apply(lambda x:' '.join(ps.stem(word) for word in x.split()))


### Feature selection

In [19]:
from nltk.tokenize import word_tokenize

all_words=[] 
for msg in processed:
    wd=word_tokenize(msg)
    for word in wd:
        all_words.append(word)
        


In [20]:
all_words=nltk.FreqDist(all_words)

In [21]:
len(all_words)

6374

In [22]:
lol=list(all_words.most_common(1500))
word_features=[]
for x in lol:
    word_features.append(x[0])
    
    


In [23]:
def find_features(msg):
    res={}
    
    for word in word_features:
        res[word]=(word in msg.split())
            
    return res


In [24]:
y=find_features(processed[0])

In [25]:
messages=list(zip(processed,label))


In [26]:
seed=1 
np.random.seed=seed 
np.random.shuffle(messages)

featuresset=[(find_features(text),Y) for (text,Y) in messages]

In [27]:
from sklearn import model_selection

training,testing=model_selection.train_test_split(featuresset,test_size=0.25,random_state=seed)

In [52]:
import pickle
with open('traindata.txt', 'wb') as filehandle:
    # store the data as binary data stream
    pickle.dump(training, filehandle)
    
with open('testdata.txt', 'wb') as filehandle:
    # store the data as binary data stream
    pickle.dump(testing, filehandle)
